In [1]:
import torch
torch.__version__

'2.3.1+cu121'

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import torch.utils.data
import math
from copy import deepcopy
import numpy as np
import cv2 as cv
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim     # for constructing optimizer
import torchvision.models as models
from module import *
from function import *

# 1. preparing the dataset
class GDdataset(Dataset):
    def __init__(self, path):
        self.data = pd.read_csv(path)
        SFR = torch.tensor(self.data.iloc[:, 2:46].values)
        blocks = [SFR[:, i:i+4].reshape(-1, 2, 2) for i in range(0, 45, 5)] # 将36列中的每4列合并成一个2*2矩阵，得到9个块
        
        self.value = torch.cat([torch.cat(blocks[i:i+3], dim=2) for i in range(0, 9, 3)], dim=1) # 将9个块按3*3的方式拼成一个大矩阵
        self.value = self.value.unsqueeze(1).to(torch.float32)
        
        self.target = torch.tensor([1.0 if x == 'OK' else 0.0 for x in self.data.iloc[:, 48].values])
        self.target = self.target.unsqueeze(1)
        
    def __getitem__(self, index):
        return self.value[index], self.target[index]
        
        
    def __len__(self):
        return len(self.data)

In [4]:
# 2. define the model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 卷积层
        # input:(batch_size, 1, 6, 6), output:(batch_size, 9, 3, 3)
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=2, stride=2),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=1),
        )
        
        # 全连接层
        self.fc = nn.Sequential(
            nn.Linear(in_features=16*3*3, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=64),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=64, out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=1),
        )
        
        self.sigmoid = nn.Sigmoid()
        
        
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.shape[0], -1)  # (batch_size, 9*3*3)
        x = self.fc(x)
        x = self.sigmoid(x)     # (batch_size, 1)
        return x
    
    
batch_size = 128
learning_rate = 0.1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

train_dataset = GDdataset("./train_data.csv")
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataset = GDdataset("./test_data.csv")
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [5]:
from torch.utils.tensorboard import SummaryWriter   
tb = SummaryWriter()

model = Net()
model.to(device)

# 3. Construct Loss and Optimizer
loss_function = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

# 4. define training cycle
def train(model, epoch):
    model.train()
    total_loss = 0
    total_correct = 0
    for batch_idx, (value, target) in enumerate(train_loader):
        value, target = value.to(device), target.to(device)    # 扔给GPU
        optimizer.zero_grad()
        # forward + backward + update
        output = model(value)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()

        predicted = (output > 0.5).float()
        total_correct += (predicted == target).sum().item()
        total_loss += loss.item()
        
        progress = math.ceil(batch_idx / len(train_loader) * 50)
        print("\rTrain epoch %d: %d/%d, [%-51s] % d%%" % (epoch, len(train_dataset), len(train_loader.dataset), '-' * progress + '>', progress * 2), end="")
        
    # 输出每轮的loss
    # print("\n\n[epoch %d] loss: %.3f train_accuracy: %d / %d=%.3f" % (epoch+1, running_loss, total_correct, len(train_dataset), total_correct / len(train_dataset)))
    
    tb.add_scalar("Loss", total_loss, epoch)              # scalar标量，即添加一个数字
    tb.add_scalar("Number Correct", total_correct, epoch)
    tb.add_scalar("Accuracy", total_correct / len(train_dataset), epoch)
    
def test(model, epoch):
    model.eval()
    with torch.no_grad(): 
        correct = 0         # 分类正确个数
        # total = 0           # 总数
        test_loss = 0
        for value, target in test_loader:
            value, target = value.to(device), target.to(device)    # 扔给GPU
            output = model(value)       # (batch_size, 1)
            predicted = (output > 0.5).float()
            # total += target.size(0)     # 加batch_size
            correct += (predicted == target).sum().item()
            test_loss += loss_function(output, target).item()
            
        test_loss /= len(test_loader.dataset)
        
        print("\nTest: average loss: {:.4f}, test_accuracy: {}/{} ({:.0f}%)".format(test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
        tb.add_scalar("Test_accuracy", correct / len(test_loader.dataset), epoch)
        
    # print("Accuracy on test set: %.1f%%" % (100 * correct / len(test_loader.dataset)))

In [6]:
for name, param in model.named_parameters():
    print(f"Parameter {name} is on device: {param.device}")

Parameter conv.0.weight is on device: cuda:0
Parameter conv.0.bias is on device: cuda:0
Parameter conv.1.weight is on device: cuda:0
Parameter conv.1.bias is on device: cuda:0
Parameter fc.0.weight is on device: cuda:0
Parameter fc.0.bias is on device: cuda:0
Parameter fc.2.weight is on device: cuda:0
Parameter fc.2.bias is on device: cuda:0
Parameter fc.5.weight is on device: cuda:0
Parameter fc.5.bias is on device: cuda:0
Parameter fc.7.weight is on device: cuda:0
Parameter fc.7.bias is on device: cuda:0


## Pre-train

In [20]:
# for epoch in range(50):
#     train(model, epoch)
#     test(model, epoch)
# torch.save(model.state_dict(), "model.pt")

## PyTorch built-in quantization

In [ ]:
model = Net().to('cpu')
model.load_state_dict(torch.load('model.pt', map_location='cpu'))
model.eval()
print('Here is the floating point version of this module:')
print(model)
qmodel = torch.quantization.quantize_dynamic(
    model, {nn.Conv2d, nn.Linear, nn.BatchNorm2d, nn.ReLU, nn.Sigmoid}, dtype=torch.qint8
)
print('and now the quantized version:')
print(qmodel)
from function import *
# compare the sizes
f=print_size_of_model(model,"fp32")
q=print_size_of_model(qmodel,"int8")
print("{0:.2f} times smaller".format(f/q))
# compare the performance
print("Floating point FP32")
# %timeit test(model, 50)
test(model, 10)
run_benchmark(model, test_loader)
print("Quantized INT8")
# %timeit test(qmodel, 10)
test(qmodel, 10)
run_benchmark(qmodel, test_loader)

## Profiler

In [8]:
import torch.profiler
from tqdm import tqdm
model = Net().to(device)
model.load_state_dict(torch.load('model.pt', map_location='cpu'))
model.eval()

profiler = torch.profiler.profile(
    schedule=torch.profiler.schedule(wait=2, warmup=2, active=6, repeat=1),
    on_trace_ready=torch.profiler.tensorboard_trace_handler(dir_name='./performance/'),
    activities=[
        torch.profiler.ProfilerActivity.CPU,
        torch.profiler.ProfilerActivity.CUDA
    ],
    with_stack=True,
)
profiler.start()
with torch.no_grad():
    # for value, target in test_loader:
    #     value, target = value.to(device), target.to(device)    # 扔给GPU
    #     output = model(value)       # (batch_size, 1)
        # predicted = (output > 0.5).float()
        # # total += target.size(0)     # 加batch_size
        # correct += (predicted == target).sum().item()
        # test_loss += loss_function(output, target).item()
    for batch_idx in tqdm(range(100), desc='Profiling ...'):
        value, _ = next(iter(train_loader))
        model(value.to(device))
        profiler.step()
profiler.stop()

Profiling ...:   0%|          | 0/100 [00:00<?, ?it/s]

Profiling ...:   1%|          | 1/100 [00:00<00:16,  5.89it/s]STAGE:2024-06-07 17:32:45 1751947:1751947 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-06-07 17:32:45 1751947:1751947 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-06-07 17:32:45 1751947:1751947 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
Profiling ...: 100%|██████████| 100/100 [00:00<00:00, 192.13it/s]
